In [2]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

22/06/30 16:25:37 WARN Utils: Your hostname, DESKTOP-EJLBN3A resolves to a loopback address: 127.0.1.1; using 172.27.7.226 instead (on interface eth0)
22/06/30 16:25:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/30 16:25:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/30 16:25:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
dailydf = spark.read.csv("../data/Daily_Country_Wise_Confirmed_Cases.csv", header=True)
timeline_df = spark.read.csv("../data/Worldwide_Case_Detection_Timeline.csv", header=True)
# dailydf.show(5)
# dailydf.describe()
# len(dailydf.columns)
timeline_df = timeline_df.fillna({"Age": "unknown", "Gender": "unknown", "Symptoms": "unknown", "Hospitalised (Y/N/NA)": "NA", "Isolated (Y/N/NA)": "NA", "Travel_History (Y/N/NA)": "NA"})
timeline_df.show(30)

+-----------------+-------------+----------+-------+-------+--------------------+---------------------+-----------------+-----------------------+
|Date_confirmation|      Country|      City|    Age| Gender|            Symptoms|Hospitalised (Y/N/NA)|Isolated (Y/N/NA)|Travel_history (Y/N/NA)|
+-----------------+-------------+----------+-------+-------+--------------------+---------------------+-----------------+-----------------------+
|       2022-05-06|      England|    London|unknown|unknown|                rash|                    Y|                Y|                      Y|
|       2022-05-12|      England|    London|unknown|unknown|                rash|                    Y|                Y|                      N|
|       2022-05-13|      England|    London|unknown|unknown|      vesicular rash|                    N|                Y|                      N|
|       2022-05-15|      England|    London|unknown|   male|      vesicular rash|                    Y|                Y|   